# 02 – Renk Benzerliği Bazlı Baseline

Bu defterde dominant renk vektörleri üzerinden basit bir benzerlik bazlı baseline kuracağız ve metrikleri hesaplayacağız.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error

df_colors = pd.read_csv('data/processed/colors.csv')
df_colors.head()

In [ ]:
# Yardımcı fonksiyonlar
def vector_from_row(row, n_colors=5):
    cols = [f'c{i}' for i in range(n_colors*3)]
    return row[cols].values.astype(np.float32)

def cosine_similarity(a, b):
    dot = np.dot(a, b)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return dot / (na * nb)

In [ ]:
# Örnek için rastgele çiftler oluşturma (gerçek projede etiketli çiftler kullanın)
import itertools
import random

indices = list(df_colors.index)
pairs = []
for _ in range(500):
    i, j = random.sample(indices, 2)
    pairs.append((i, j))

len(pairs)

In [ ]:
# Basit benzerlik skoru ve pseudo-label üretimi (örnek)
sims = []
labels = []

for i, j in pairs:
    v1 = vector_from_row(df_colors.loc[i])
    v2 = vector_from_row(df_colors.loc[j])
    s = cosine_similarity(v1, v2)
    sims.append(s)
    # Örnek pseudo-label: similarity > 0.8 -> uyumlu
    labels.append(1 if s > 0.8 else 0)

sims = np.array(sims)
labels = np.array(labels)
labels[:10], sims[:10]

In [ ]:
# Eşik bazlı tahmin ve metrikler
threshold = 0.8
preds = (sims > threshold).astype(int)

acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds)
try:
    auc = roc_auc_score(labels, sims)
except ValueError:
    auc = float('nan')

rmse = mean_squared_error(labels, sims, squared=False)
mae = mean_absolute_error(labels, sims)

metrics = {
    'accuracy': acc,
    'f1': f1,
    'roc_auc': auc,
    'rmse': rmse,
    'mae': mae,
}
metrics

In [ ]:
# Sonuçları kaydet
import json, os
os.makedirs('results/baseline', exist_ok=True)
with open('results/baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print('Kaydedildi: results/baseline/metrics.json')